In [ ]:
import os
import glob
import sys
import numpy as np
from datetime import datetime
from multiprocessing import Pool
from tqdm import tqdm
import gc

from lmsiage.mesh_file import MeshFile
from lmsiage.zarr_index import get_file_arrays
from lmsiage.zarr_index_tools import update_index_for_dir

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
force = False
mesh_dir = 'zarr/mesh'
age_dir = 'zarr/age'
update_index_for_dir(age_dir)

mesh_files = sorted(glob.glob(f'{mesh_dir}/*/*.zip'))
dst_datetimes = [datetime.strptime(os.path.basename(f), 'mesh_%Y%m%d.zip') for f in mesh_files]
print(len(dst_datetimes), dst_datetimes[0], dst_datetimes[-1])

In [ ]:
for dst_datetime in tqdm(dst_datetimes):    
    age_file = dst_datetime.strftime(f'{age_dir}/%Y/age_%Y%m%d.zip')
    mesh_file = dst_datetime.strftime(f'{mesh_dir}/%Y/mesh_%Y%m%d.zip')

    age_arrays = get_file_arrays(age_file)
    if len(age_arrays) == 0:
        continue
    
    if 'age' in get_file_arrays(age_file) and not force :
        continue

    mf = MeshFile(mesh_file)
    sic = mf.load(['sic'], as_dict=False)[0]

    mf = MeshFile(age_file)
    src_sic_names = sorted([sic_name for sic_name in age_arrays if 'sic' in sic_name])
    src_sic = mf.load(src_sic_names, as_dict=False)
    fractions = np.diff(np.array([np.zeros_like(sic)] + src_sic + [sic]), axis=0)
    nfracs = fractions.shape[0]

    if datetime(4, dst_datetime.month, dst_datetime.day) >= datetime(4, 9, 15):
        dst_age_offset = dst_datetime - datetime(dst_datetime.year, 9, 15)
    else:
        dst_age_offset = dst_datetime - datetime(dst_datetime.year-1, 9, 15)
    dst_age_offset = dst_age_offset.days / 365.

    years = np.arange(nfracs, 0, -1)
    years = years - 1 + dst_age_offset
    age = np.sum(fractions * years[None].T, axis=0)/100.

    data = {'age': age.astype(np.float32), 'f': fractions.astype(np.float32)}
    mf.save(data, mode='o')


In [ ]:
date = datetime(1999, 12, 31)
x, y, t = MeshFile(date.strftime(f'{mesh_dir}/%Y/mesh_%Y%m%d.zip')).load(['x', 'y', 't'], as_dict=False)
age, fracs = MeshFile(date.strftime(f'{age_dir}/%Y/age_%Y%m%d.zip')).load(['age', 'f'], as_dict=False)

In [ ]:
plt.tripcolor(x, y, t, age, cmap='jet')
plt.colorbar()

In [ ]:
age_files = sorted(glob.glob(f'{age_dir}/*/age_????????.zip'))[::20]
age_indices = list(range(len(age_files)))
print(len(age_files), len(age_indices), age_files[0], age_files[-1])

In [ ]:
cmap = 'plasma'
vmin = 0
vmax = 4

def plot_age(i, age_file):
    mesh_file = age_file.replace('age_', 'mesh_').replace('/age/', '/mesh/')
    dst_frame_name = f'{age_dir}/frames/frame_{i:04}.png'
    
    age = MeshFile(age_file).load(['age'], as_dict=False)[0]
    x, y, t = MeshFile(mesh_file).load(['x', 'y', 't'], as_dict=False)

    fig = plt.figure(figsize=(10,10))
    plt.tripcolor(x, y, age, triangles=t, cmap=cmap, vmin=vmin, vmax=vmax)
    plt.colorbar(shrink=0.5)
    date_str = os.path.basename(age_file).split('_')[1].split('.')[0]
    text = f'#{i:04} {date_str[:4]}-{date_str[4:6]}-{date_str[6:8]}'
    plt.text(-2000, 2200, text, fontsize=16)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(dst_frame_name, bbox_inches='tight', pad_inches=0, facecolor='white')
    plt.close('all')
    plt.close()
    gc.collect()


for i, age_file in tqdm(zip(age_indices, age_files), total=len(age_files)):
    plot_age(i, age_file)